In [10]:
# Importa bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import time
from joblib import Parallel, delayed

import numpy as np


In [11]:
# Leitura de dados
df = pd.read_csv("../data/raw/creditcard.csv")

# Exploração de dados e pré-processamento
print(df.describe())

# Seleciona apenas variáveis numéricas
df = df.select_dtypes(include=["int64", "float64"])
df.drop(["Time", "Amount"], axis=1, inplace=True)

# Normaliza os dados
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Separação de treino e teste
X_train, X_test, y_train, y_test = train_test_split(df_scaled, df["Class"], test_size=0.25, random_state=1512)

# Oversampling e undersampling
over = RandomOverSampler(sampling_strategy="auto")
under = RandomUnderSampler(sampling_strategy="auto")

# Use o índice para acessar o conjunto de dados resamplenado
X_train_s = over.fit_resample(X_train, y_train)[0]
y_train_s = over.fit_resample(X_train, y_train)[1]

# Use o índice para acessar o conjunto de dados resamplenado novamente
X_train_s = under.fit_resample(X_train_s, y_train_s)[0]
y_train_s = under.fit_resample(X_train_s, y_train_s)[1]

# Modelagem em paralelo
models = [
    ("SVM", SVC(kernel="rbf")),
    ("KNN", KNeighborsClassifier(n_neighbors=5)),
    ("Random Forest", RandomForestClassifier(n_estimators=100)),
]

def train_model_parallel(name, model, X_train_s, y_train_s, X_test, y_test):
    # Inicia o cronômetro
    start_time = time.time()
    
    # Treina o modelo
    model.fit(X_train_s, y_train_s)
    
    # Para o cronômetro
    end_time = time.time()
    time_taken = end_time - start_time

    # Prediz as classes de teste
    y_pred = model.predict(X_test)

    # Calcula a sensibilidade e especificidade do modelo
    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    sensitivity = cm[0, 0] / (cm[0, 0] + cm[1, 0])
    specificity = cm[1, 1] / (cm[1, 1] + cm[0, 1])
    
    # Imprime os resultados na tela
    print(f"Resultados para o modelo {name}:")
    print(f"Tempo de execução (s): {time_taken}")
    print(f"Sensibilidade (%): {sensitivity * 100}")
    print(f"Especificidade (%): {specificity * 100}")
    
    # Retorna os resultados
    return {
        "Modelo": name,
        "Tempo de execução (s)": time_taken,
        "Sensibilidade (%)": sensitivity * 100,
        "Especificidade (%)": specificity * 100,
    }

# Modelagem em paralelo
results_parallel = Parallel(n_jobs=-1)(
    delayed(train_model_parallel)(name, model, X_train_s, y_train_s, X_test, y_test)
    for name, model in models
)

# Convertendo explicitamente o resultado em uma lista
results_list = list(results_parallel)

# Cria um DataFrame com os resultados
decision_matrix_parallel = pd.DataFrame(results_list)

# Imprime os resultados
print("*** Resultados TOPSIS ***")
print("Modelo | Posição")

# Definindo criteria objectives
minmax = ["min", "max", "max"]

# Definindo criteria weights
weights = [0.2, 0.6, 0.2]

# Normalizando a matriz de decisão
norm_decision = decision_matrix_parallel.copy()

# Seleciona apenas as colunas numéricas para a normalização
numeric_columns = norm_decision.select_dtypes(include=['number']).columns
norm_decision[numeric_columns] = norm_decision[numeric_columns].apply(lambda x: x / np.linalg.norm(x), axis=0)

# Calculando as distâncias ao ideal positivo e negativo
pos_ideal = norm_decision.max(axis=0)
neg_ideal = norm_decision.min(axis=0)

# Calculando a distância de cada alternativa ao ideal positivo e negativo
distance_pos = np.sqrt(np.sum((norm_decision[numeric_columns] - pos_ideal[numeric_columns]) ** 2, axis=1))
distance_neg = np.sqrt(np.sum((norm_decision[numeric_columns] - neg_ideal[numeric_columns]) ** 2, axis=1))

# Calculando o indicador TOPSIS
topsis_scores = distance_neg / (distance_pos + distance_neg)

# Classificando as alternativas de acordo com o indicador TOPSIS
ranking = topsis_scores.argsort()[::-1]

# Imprimindo os resultados
print("*** Resultados TOPSIS ***")
print("Modelo | Posição")
for i in range(len(ranking)):
    print(f"{decision_matrix_parallel.loc[ranking[i], 'Modelo']} | {i + 1}")


                Time            V1            V2            V3            V4  \
count  284807.000000  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean    94813.859575  1.168375e-15  3.416908e-16 -1.379537e-15  2.074095e-15   
std     47488.145955  1.958696e+00  1.651309e+00  1.516255e+00  1.415869e+00   
min         0.000000 -5.640751e+01 -7.271573e+01 -4.832559e+01 -5.683171e+00   
25%     54201.500000 -9.203734e-01 -5.985499e-01 -8.903648e-01 -8.486401e-01   
50%     84692.000000  1.810880e-02  6.548556e-02  1.798463e-01 -1.984653e-02   
75%    139320.500000  1.315642e+00  8.037239e-01  1.027196e+00  7.433413e-01   
max    172792.000000  2.454930e+00  2.205773e+01  9.382558e+00  1.687534e+01   

                 V5            V6            V7            V8            V9  \
count  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean   9.604066e-16  1.487313e-15 -5.556467e-16  1.213481e-16 -2.406331e-15   
std    1.380247e+00  1.332271e+00  1.23709